<a href="https://colab.research.google.com/github/aidyai/rss-ai/blob/main/rss.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
%pip install --upgrade --quiet  apify-client langchain-openai langchain
!pip install feedparser newspaper3k openai
!pip install openai==0.28
!pip install replicate

In [2]:
from langchain.indexes import VectorstoreIndexCreator
from langchain_community.utilities import ApifyWrapper
from langchain_core.documents import Document

import feedparser
import requests
from bs4 import BeautifulSoup
import openai
from newspaper import Article

In [34]:
import os
os.environ["APIFY_API_TOKEN"] = "apify_api_r2J5EYAZ2qOeCTSBDqN3RJ2IraTke911bWjC"
os.environ["OPENAI_API_KEY"] = ""


## ARTICLE EXTRACTION: This code only extracts onearticle but can be modified to work for the latest 10, 50 or 20 articles

In [37]:
# RSS feed URL
rss_feed_url = 'https://news.google.com/rss/search?q=akwa%20ibom&hl=en-US&gl=US&ceid=US%3Aen'

# Parse the RSS feed
feed = feedparser.parse(rss_feed_url)

# Get the latest just one articles
latest_articles = feed.entries[19:20]   # we just yse one for speed

In [38]:
apify = ApifyWrapper()

In [ ]:
title = []
urls_to_crawl = []

for article in latest_articles:

    # extracting the title and article URL
    title = article.title
    article_url = article.link



    # Appending the article title and URL
    #title.append({"title": title})
    urls_to_crawl.append({"url": article_url})


#this functions looks into the urls_to_crawl and extracts its text
loader = apify.call_actor(
    actor_id="apify/website-content-crawler",
    run_input={"startUrls": urls_to_crawl},
    dataset_mapping_function=lambda item: Document(
        page_content=item["text"] or "", metadata={"source": item["url"]}
    ),
)

# Extracting the title, article_url and article_text for the latest 10 articles

In [ ]:
print(title)                      #EXTRACTED TITLE
print(urls_to_crawl)              #EXTRACTED URL

article_text = loader.load()
print(article_text)               #EXTRACTED ARTICLE TEXY

In [12]:
import json
# Extracting content from the list
extracted_content = [document.page_content for document in article_text]

['NNPC Exploration and Production Limited (NNPC E&P Ltd.), NNPC Ltd.’s flagship upstream subsidiary, and Natural Oilfield Services Ltd. (NOSL), a subsidiary of Sterling Oil Exploration & Energy Production Company Ltd. (SEEPCO), have announced the successful commencement of oil production at Oil Mining Lease (OML) 13 in Akwa Ibom State, Nigeria.\nThis was revealed by the Chief Corporate Communications Officer, NNPC Ltd, Olufemi Soneye.\nIt reads,” The production, which commenced on the 6th of May 2024, with 6,000 barrels of oil, is expected to be ramped up to 40,000 barrels per day by May 27th, 2024.\n“The first oil flow from OML 13 is a historic milestone in the partnership between NNPC E&P Ltd and NOSL, highlighting their dedication to driving growth and development in Nigeria’s oil and gas sector, which remains a vital component of the nation’s economy.\n“The achievement does not only signify the culmination of rigorous planning and execution by the teams involved but also represents

In [13]:
import openai
# Send the content to OpenAI to get topic and category tags
response = openai.Completion.create(
    model="gpt-3.5-turbo-instruct",
    prompt=extracted_content,
    max_tokens=50,  # Adjust as needed
    n=1,  # Number of completions to generate
    stop=["\n"],  # Stop completion at the end of the text
    temperature=0.5,  # Adjust as needed
    top_p=1.0,
    frequency_penalty=0.0,
    presence_penalty=0.0,
    logit_bias={},
    logprobs=0,
    echo=True  # Print the input prompt along with the completion
)


# USING REPLICATE API TO CALL POWERFUL LARGE LANGUAGE MODELS

In [17]:
import replicate

In [26]:
import os
import replicate
from replicate.client import Client

replicate = Client(api_token="")

## USING MISTRAL 8x7B via Replicate Api

In [31]:
for event in replicate.stream(
    "mistralai/mixtral-8x7b-instruct-v0.1",
    input={
        "top_k": 50,
        "top_p": 0.9,
        "prompt": f"Generate 5 'TOPIC TAGS' AND 'CATEGPRY TAGS' for the given article\ {extracted_content}",
        "temperature": 0.6,
        "system_prompt": "YOU ARE AN AI ASSIATAT THAT CAN GENERATE topic tags AND  article category tags",
        "length_penalty": 1,
        "max_new_tokens": 1024,
        "prompt_template": "<s>[INST] {prompt} [/INST] ",
        "presence_penalty": 0
    },
):
    print(str(event), end="")

1. Topic Tags: Oil Production, NNPC, Natural Oilfield Services Ltd, OML 1

## USING METAS LLAMA 3 Via Replicate Api

In [33]:
for event in replicate.stream(
    "meta/meta-llama-3-70b-instruct",
    input={
        "top_k": 0,
        "top_p": 0.9,
        "prompt": f"Generate 5 'TOPIC TAGS' AND 'CATEGPRY TAGS' for the given article\ {extracted_content}",
        "max_tokens": 650,
        "min_tokens": 0,
        "temperature": 0.6,
        "system_prompt": "YOU ARE AN AI ASSITANT THAT CAN GENERATE topic tags AND article category tags",
        "length_penalty": 1,
        "stop_sequences": "<|end_of_text|>,<|eot_id|>",
        "prompt_template": "<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nYou are a helpful assistant<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n{prompt}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n",
        "presence_penalty": 1.15,
        "log_performance_metrics": False
    },
):
    print(str(event), end="")

Here are five topic tags and five category tags for the given article:

**Topic Tags:**

1. Oil Production
2. Nigeria Energy Sector
3. NNPC E&P Ltd
4. Natural Oilfield Services Ltd
5. Oil Mining Lease (OML) 13

**Category Tags:**

1. Business
2. Energy
3. Nigeria News
4. Oil and Gas
5. Economy